In [1]:
from ufal.udpipe import Model, Pipeline
import torch
import torch.nn as nn
import re

In [2]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size)  
        self.relu = nn.ReLU()                          
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.sigm = nn.Sigmoid()
    
    def forward(self, x):                              
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigm(out)
        return out
    
class DLTextClassifier (object):

    def __init__ (self, sourceForInf, sourceForDict, sourceForDL):
        self.nomberOfKnownWords = 0
        self.modelForInfinitive = Model.load(sourceForInf)
        self.vectorOfStr = torch.zeros(2000)
        self.sourceForDL = sourceForDL
        self.net = Net(2000, 1000, 42)
        self.net.load_state_dict(torch.load(sourceForDL))
        self.needToSearch = False
        
        f = open(sourceForDict, 'r')
        self.clasterOfWord = {}
        for strings in f:
            strings = strings.split(' ')
            self.clasterOfWord.update({strings[0]: int(strings[1])})
        f.close()
        
        self.idOfThemes = {'авто/мото': 0,'активный отдых': 1,'бизнес': 2,'домашние животные': 3,'здоровье': 4,
                           'знакомство и общение': 5,'игры': 6,'ИТ (компьютеры и софт)': 7,'кино': 8,
                           'красота и мода': 9,'кулинария': 10,'культура и искусство': 11,'литература': 12,
                           'мобильная связь и интернет': 13,'музыка': 14,'наука и техника': 15,'недвижимость': 16,
                           'новости и СМИ': 17,'безопасность':18,'образование':19,'обустройство и ремонт': 20,
                           'политика': 21,'продукты питания': 22,'промышленность': 23,'путешествия':24,
                           'работа': 25,'развлечения': 26,'религия': 27,'дом и семья': 28,'спорт': 29,
                           'страхование': 30,'телевидение': 31,'товары и услуги': 32,'увлечения и хобби': 33,
                           'финансы': 34,'фото': 35,'эзотерика': 36,'электроника и бытовая техника': 37,
                           'эротика': 38,'юмор': 39,'общество, гуманитарные науки': 40,'дизайн и графика': 41}
    
    def tryToUpgradeSelfVec (self, word):
        try:
            relationID = self.clasterOfWord[word.lower()]
            self.vectorOfStr[relationID] += 1
            self.nomberOfKnownWords += 1
        except Exception:
            if not re.search(r'[\W]', word):
                relationID = self.clasterOfWord[self.wordToInf(text=word)]
                self.vectorOfStr[relationID] += 1
                self.nomberOfKnownWords += 1
            else: 
                raise
    
    def wordToInf(self, text):
        process_pipeline = Pipeline(self.modelForInfinitive, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
        wordInfo = process_pipeline.process(text).split('\n')[4].split('\t')
        if (wordInfo[3] == 'NUM'):
            return ('_NUM_' + ('x' * len(wordInfo[2])))
        else:
            return wordInfo[2]
    
    def text2Vec (self, text):
        self.vectorOfStr = torch.zeros(2000)
        self.nomberOfKnownWords = 0
        words = re.findall(r'[0-9A-Za-zА-Яа-я-.]+', text)
        for word in words:
            try :
                self.tryToUpgradeSelfVec(word)
            except Exception:
                for singleWord in re.findall(r"[\w']+", word):
                    try: 
                        self.tryToUpgradeSelfVec(singleWord)
                    except Exception:
                        if (self.needToSearch == True):
                            url = 'https://rusvectores.org/araneum_none_fasttextcbow_300_5_2018/' + singleWord + '/api/json/'
                            #url = 'https://rusvectores.org/tayga_none_fasttextcbow_300_10_2019/' + singleWord + '/api/json/'
                            response = requests.get(url)
                            for i in range (10):
                                try:
                                    sinonim = str(response.json())
                                    sinonim = sinonim.split("'")[5 + i * 2].split('_')[0]
                                    relationID = int(self.clasterOfWord[sinonim.lower()])
                                    self.clasterOfWord.update({singleWord: relationID})
                                    self.vectorOfStr[relationID] = self.vectorOfStr[relationID] + 1
                                    self.nomberOfKnownWords = self.nomberOfKnownWords + 1
                                    break
                                except Exception:
                                    continue
                                                 
        self.vectorOfStr = self.vectorOfStr / self.nomberOfKnownWords  
        return self.vectorOfStr
    
    def updateDLNetwork (self, text, themes, cicles=5, learing_rate=0.001):
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.net.parameters(), lr=learing_rate)
        
        newLabel = torch.zeros (len(themes), dtype=torch.long)
        for i in range (len(themes)):
            newLabel[i] = self.idOfThemes[themes[i]]
        
        newData = torch.zeros(len(themes), 2000)
        vecOfText = self.text2Vec(text)
        for i in range (2000):
            for j in range (len(themes)):
                newData[j][i] = vecOfText[i]

        for i in range (cicles):    
            labels = newLabel
            optimizer.zero_grad()                             
            outputs = self.net(newData)                             
            loss = criterion(outputs, newLabel)                 
            loss.backward()                                   
            optimizer.step()  
            
        torch.save(self.net.state_dict(), self.sourceForDL)
        
    def themesOfTheText (self, text, themes):
        vecOfText = self.text2Vec(text)
        vecOfRelation = self.net(vecOfText)
        returnData = {}
        for i in range ( len(themes) ):
            index = self.idOfThemes[themes[i]]
            returnData.update({themes[i]: int ( 100 * float(vecOfRelation[index]))})
        return returnData

In [3]:
allThemes = ['авто/мото', 'активный отдых','бизнес','домашние животные','здоровье','знакомство и общение','игры',
             'ИТ (компьютеры и софт)','кино','красота и мода','кулинария','культура и искусство','литература',
             'мобильная связь и интернет','музыка','наука и техника','недвижимость','новости и СМИ','безопасность',
             'образование','обустройство и ремонт','политика','продукты питания','промышленность','путешествия',
             'работа','развлечения','религия','дом и семья','спорт','страхование','телевидение','товары и услуги',
             'увлечения и хобби','финансы','фото','эзотерика','электроника и бытовая техника','эротика','юмор',
             'общество, гуманитарные науки','дизайн и графика']

In [8]:
textClassifier = DLTextClassifier('udpipe_syntagrus.model', 'dict_with_search.txt', 'neurotext_with_vec.pkl')

In [9]:
textClassifier.themesOfTheText('Инвестиционный фонд стал владельцем пакета акций в аптечной сети', allThemes)

{'авто/мото': 0,
 'активный отдых': 0,
 'бизнес': 0,
 'домашние животные': 2,
 'здоровье': 0,
 'знакомство и общение': 0,
 'игры': 0,
 'ИТ (компьютеры и софт)': 0,
 'кино': 0,
 'красота и мода': 0,
 'кулинария': 0,
 'культура и искусство': 0,
 'литература': 0,
 'мобильная связь и интернет': 0,
 'музыка': 0,
 'наука и техника': 0,
 'недвижимость': 0,
 'новости и СМИ': 0,
 'безопасность': 0,
 'образование': 0,
 'обустройство и ремонт': 0,
 'политика': 0,
 'продукты питания': 0,
 'промышленность': 0,
 'путешествия': 0,
 'работа': 0,
 'развлечения': 0,
 'религия': 0,
 'дом и семья': 0,
 'спорт': 0,
 'страхование': 3,
 'телевидение': 0,
 'товары и услуги': 1,
 'увлечения и хобби': 0,
 'финансы': 0,
 'фото': 0,
 'эзотерика': 0,
 'электроника и бытовая техника': 0,
 'эротика': 0,
 'юмор': 0,
 'общество, гуманитарные науки': 0,
 'дизайн и графика': 0}

In [11]:
textClassifier.updateDLNetwork('Инвестиционный фонд стал владельцем пакета акций в аптечной сети', ['финансы', 'новости и СМИ', 'бизнес'], cicles=30)
textClassifier.themesOfTheText('Инвестиционный фонд стал владельцем пакета акций в аптечной сети', allThemes)

{'авто/мото': 0,
 'активный отдых': 0,
 'бизнес': 91,
 'домашние животные': 0,
 'здоровье': 0,
 'знакомство и общение': 0,
 'игры': 0,
 'ИТ (компьютеры и софт)': 0,
 'кино': 0,
 'красота и мода': 0,
 'кулинария': 0,
 'культура и искусство': 0,
 'литература': 0,
 'мобильная связь и интернет': 0,
 'музыка': 0,
 'наука и техника': 0,
 'недвижимость': 0,
 'новости и СМИ': 94,
 'безопасность': 0,
 'образование': 0,
 'обустройство и ремонт': 0,
 'политика': 0,
 'продукты питания': 0,
 'промышленность': 0,
 'путешествия': 0,
 'работа': 0,
 'развлечения': 0,
 'религия': 0,
 'дом и семья': 0,
 'спорт': 0,
 'страхование': 0,
 'телевидение': 0,
 'товары и услуги': 0,
 'увлечения и хобби': 0,
 'финансы': 99,
 'фото': 0,
 'эзотерика': 0,
 'электроника и бытовая техника': 0,
 'эротика': 0,
 'юмор': 0,
 'общество, гуманитарные науки': 0,
 'дизайн и графика': 0}